# **Gujarati Character Recognition**

## Drive Mount

In [4]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


### Data Copy

In [0]:
# %rm data.tar.xz
# %rm -rf dataAll
%cp /gdrive/'My Drive'/projectdata/data.tar.xz /content/

In [0]:
import tarfile
with tarfile.open('data.tar.xz') as f:
    f.extractall('.')

In [7]:
%ls dataAll

0/   11/  14/  17/  2/   22/  25/  28/  30/  33/  36/  39/  41/  44/  5/  8/
1/   12/  15/  18/  20/  23/  26/  29/  31/  34/  37/  4/   42/  45/  6/  9/
10/  13/  16/  19/  21/  24/  27/  3/   32/  35/  38/  40/  43/  46/  7/


## **Imports**

In [8]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from random import shuffle
import pickle

import keras
from keras.utils import to_categorical
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


### Global Variables

In [0]:
IMG_SIZE = 150
LR = 4e-3
batch_size = 32
num_classes = 47
epochs = 20
data_augmentation = False
num_predictions = 20
model_name = 'trained_model.h5'

# Data Preprocessing



In [11]:
#%rm train_data.dat
path = "dataAll"

def create_train_data():
  training_data = []

  if os.path.exists("train_data.dat"):
    file = open('train_data.dat', 'rb')
    training_data = pickle.load(file)
    file.close()
    return training_data

  #img_count = 0
  for folder in tqdm(os.listdir(path)):
    p = path + "/" + folder
    files = os.listdir(p)
    for i in files:
      label = folder
      img_loc = p + "/" + i
      img = cv2.imread(img_loc,cv2.IMREAD_GRAYSCALE)
      img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
      training_data.append([np.array(img),np.array(label)])
      #img_count += 1
      #if img_count > 6000 : break

  shuffle(training_data)

  file = open('train_data.dat', 'wb')
  pickle.dump(training_data, file)  
  file.close()

  return training_data

train_data = create_train_data()
%ls

100%|██████████| 47/47 [00:09<00:00,  4.72it/s]


dataAll/  data.tar.xz  sample_data/  train_data.dat


In [12]:
%cp /content/train_data.dat /gdrive/'My Drive'/projectdata/
%ls /gdrive/'My Drive'/projectdata

'Copy of Gujarati ML.ipynb'                       train_data.dat
'Copy of ML assign using Keras preprocessed.7z'   Untitled0.ipynb
 data.tar.xz


## Data Split

In [13]:
train = train_data[:-6000]
test = train_data[-6000:]
# Training Data
x_train = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
y_train = [i[1] for i in train]

y_train = to_categorical(y_train,47)
print(y_train)

# Testing Data
x_test = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)

y_test = [i[1] for i in test]
print(y_test)
y_test = to_categorical(y_test,47)
print(y_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[array('28', dtype='<U2'), array('11', dtype='<U2'), array('41', dtype='<U2'), array('11', dtype='<U2'), array('21', dtype='<U2'), array('32', dtype='<U2'), array('10', dtype='<U2'), array('26', dtype='<U2'), array('18', dtype='<U2'), array('40', dtype='<U2'), array('25', dtype='<U2'), array('14', dtype='<U2'), array('6', dtype='<U1'), array('44', dtype='<U2'), array('30', dtype='<U2'), array('5', dtype='<U1'), array('7', dtype='<U1'), array('32', dtype='<U2'), array('8', dtype='<U1'), array('43', dtype='<U2'), array('39', dtype='<U2'), array('34', dtype='<U2'), array('8', dtype='<U1'), array('33', dtype='<U2'), array('17', dtype='<U2'), array('5', dtype='<U1'), array('38', dtype='<U2'), array('32', dtype='<U2'), array('38', dtype='<U2'), array('16', dtype='<U2'), array('43', dtype='<U2'), array('7', dtype='<U1'), array('7', dtype='

## Design Model

In [14]:
model = Sequential()
model.add(Conv2D(128, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
print(x_train.shape[1:])
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# Initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)




(150, 150, 1)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Model Train

In [15]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)



Not using data augmentation.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 27735 samples, validate on 6000 samples
Epoch 1/20





27735/27735 [==============================] - 83s 3ms/step - loss: 2.9767 - acc: 0.2231 - val_loss: 2.0307 - val_acc: 0.4820
Epoch 2/20
27735/27735 [==============================] - 67s 2ms/step - loss: 1.8642 - acc: 0.4834 - val_loss: 1.3384 - val_acc: 0.6490
Epoch 3/20
27735/27735 [==============================] - 67s 2ms/step - loss: 1.3227 - acc: 0.6221 - val_loss: 1.0467 - val_acc: 0.7157
Epoch 4/20
27735/27735 [==============================] - 67s 2ms/step - loss: 1.0260 - acc: 0.7054 - val_loss: 0.9667 - val_acc: 0.7397
Epoch 5/20
27735/27735 [==============================] - 67s 2ms/step - loss: 0.8314 - acc: 0.7571 - val_loss: 0.8447 - val_acc: 0.7625
Epoch 6/20
27735/27735 [==============================] - 67s 2ms/step - loss: 0.7070 - acc: 0.7944 - val_loss: 0.7996 - val_acc: 0.7

## Results

In [16]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

6000/6000 [==============================] - 4s 637us/step
Test loss: 0.6224369748036067
Test accuracy: 0.838


## Saving the model

In [17]:
# Save model and weights
model.save(model_name)
%cp /content/trained_model.h5 /gdrive/'My Drive'/projectdata/
%ls /gdrive/'My Drive'/projectdata

'Copy of Gujarati ML.ipynb'                       train_data.dat
'Copy of ML assign using Keras preprocessed.7z'   trained_model.h5
 data.tar.xz                                      Untitled0.ipynb
